<a href="https://colab.research.google.com/github/danielahernandez29/Curso-IA/blob/main/CNN_IA/Clasificaci%C3%B3n_de_Ropa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
El código muestra una red convolucional (CNN) para clasificar prendas de ropa
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

# Cargo el dataset de Fashion MNIST
fashion_mnist = tf.keras.datasets.fashion_mnist

# Descargo los datos de entrenamiento y prueba
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Defino los nombres de las clases
class_names = ['Camiseta/top', 'Pantalón', 'Suéter', 'Vestido', 'Abrigo',
               'Sandalia', 'Camisa', 'Zapatilla deportiva', 'Bolso', 'Botín']


index = 10
image = train_images[index]
label = train_labels[index]

print(image)

# Visualizo la imagen en escala de grises
plt.figure(figsize=(3, 3))
plt.imshow(image, cmap=plt.cm.gray)
plt.title(f'Etiqueta número: {label} es {class_names[label]}')
plt.show()

# Verifico las dimensiones de los conjuntos
print(train_images.shape)
print(test_images.shape)

# Normalizo las imágenes
train_images = train_images / 255.0
test_images = test_images / 255.0


train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# Creo el modelo de red convolucional
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10)  # 10 salidas porque hay 10 clases
])

# Defino la tasa de aprendizaje
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)

# Compilo el modelo con su optimizador y función de pérdida
model.compile(optimizer=adam_optimizer,
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entreno el modelo con las imágenes de entrenamiento
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# Evalúo el modelo con los datos de prueba
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Precisión en prueba: {test_acc:.2f}")

# Creo un modelo de probabilidad
probability_model = Sequential([model, Softmax()])
predictions = probability_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Calculo la matriz de confusión
cm = confusion_matrix(test_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# Grafico la matriz de confusión
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(ax=ax, cmap='Blues')
plt.title("Matriz de Confusión")
plt.show()

# Función para graficar imagen + predicción
def plot_image(i, predictions_array, true_labels, imgs):
    true_label, img = true_labels[i], imgs[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img.reshape((28, 28)), cmap=plt.cm.binary)

    predicted_label = tf.argmax(predictions_array).numpy()
    color = 'blue' if predicted_label == true_label else 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(
        class_names[predicted_label],
        100 * tf.reduce_max(predictions_array).numpy(),
        class_names[true_label]),
        color=color)

# Muestro algunas imágenes con sus predicciones
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], test_labels, test_images)
plt.tight_layout()
plt.show()
